In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions

pd.set_option('display.max_rows', None)


In [2]:
model_file = 'saved/SASRec-Dec-05-2021_08-51-44.pth'

checkpoint = torch.load(model_file)
config = checkpoint['config']
# config.final_config_dict['load_col']['inter'].append('item_type_recommend')
config.final_config_dict['unused_col']['inter'] = ['exposure_time', 'exposure_time_list']
# config.final_config_dict['save_dataset'] = True
# config.final_config_dict['save_dataloaders'] = True

init_seed(config['seed'], config['reproducibility'])
init_logger(config)
logger = getLogger()
logger.info(config)

dataset = create_dataset(config)
logger.info(dataset)
train_data, valid_data, test_data = data_preparation(config, dataset)

init_seed(config['seed'], config['reproducibility'])
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
model.load_state_dict(checkpoint['state_dict'])
model.load_other_parameter(checkpoint.get('other_parameter'))

05 Dec 09:11    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/pretrain
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = {'uniform': 1}
eval_step = 1
stopping_step = 500
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'LS': 'valid_and_test'}, 'order': 'TO', 'mode': 'full', 'group_by': 'user'}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = exposure_time


In [3]:
trainer = get_trainer(config['MODEL_TYPE'], config['model'])(config, model)
test_result = trainer.evaluate(test_data, load_best_model=False, show_progress=config['show_progress'])
print(test_result)

Evaluate   :   0%|                                                            | 0/1 [00:00<?, ?it/s]:   0%|                                   | 0/1 [00:00<?, ?it/s, GPU RAM: 0.76 G/79.35 G]: 100%|███████████████████████████| 1/1 [00:00<00:00, 23.82it/s, GPU RAM: 0.76 G/79.35 G]


OrderedDict([('recall@10', 0.2127), ('mrr@10', 0.1271), ('ndcg@10', 0.1472), ('hit@10', 0.2127), ('precision@10', 0.0213)])


In [4]:
# item_type_uid = test_data.dataset.inter_feat[test_data.uid_field][test_data.dataset.inter_feat['item_type_recommend'] == 1]
# item_type_data = copy.copy(test_data)
# item_type_data.dataset = copy.copy(test_data.dataset)
# if test_data.is_sequential:
#     _, index = (test_data.dataset.inter_feat[test_data.uid_field] == item_type_uid[:, None]).nonzero(as_tuple=True)
#     item_type_data.dataset.inter_feat = item_type_data.dataset.inter_feat[index]
# else:
#     item_type_data.uid_list = item_type_uid
#     item_type_data.user_df = dataset.join({test_data.uid_field: item_type_uid})
# item_type_result = trainer.evaluate(item_type_data, load_best_model=False, show_progress=config['show_progress'])
# print(item_type_result)

In [5]:
item_type_list = torch.unique(test_data.dataset.inter_feat['item_type_recommend'])
print(item_type_list)
item_type_res_dict = {
    'item_type_id': [],
    'total_inter_num': [],
    'train_inter_num': [],
    'valid_inter_num': [],
    'test_inter_num': [],
    'recall@10': [],
    'mrr@10': [],
    'ndcg@10': [],
    'hit@10': [],
    'precision@10': [],
}
for item_type_id in tqdm(item_type_list.numpy()):
    index = test_data.dataset.inter_feat['item_type_recommend'] == item_type_id
    item_type_uid = torch.unique(test_data.dataset.inter_feat[test_data.uid_field][index])
    item_type_data = copy.copy(test_data)
    item_type_data.dataset = copy.copy(test_data.dataset)
    if test_data.is_sequential:
        item_type_data.dataset.inter_feat = item_type_data.dataset.inter_feat[index]
    else:
        item_type_data.uid_list = item_type_uid
        item_type_data.user_df = dataset.join({test_data.uid_field: item_type_uid})
    item_type_result = trainer.evaluate(item_type_data, load_best_model=False, show_progress=False)
    item_type_res_dict['item_type_id'].append(item_type_id)
    item_type_res_dict['total_inter_num'].append((dataset.inter_feat['item_type_recommend'] == item_type_id).sum().item())
    item_type_res_dict['train_inter_num'].append((train_data.dataset.inter_feat['item_type_recommend'] == item_type_id).sum().item())
    item_type_res_dict['valid_inter_num'].append((valid_data.dataset.inter_feat['item_type_recommend'] == item_type_id).sum().item())
    item_type_res_dict['test_inter_num'].append(index.sum().item())
    item_type_res_dict['recall@10'].append(item_type_result['recall@10'])
    item_type_res_dict['mrr@10'].append(item_type_result['mrr@10'])
    item_type_res_dict['ndcg@10'].append(item_type_result['ndcg@10'])
    item_type_res_dict['hit@10'].append(item_type_result['hit@10'])
    item_type_res_dict['precision@10'].append(item_type_result['precision@10'])
    # print(item_type_id, item_type_result)
    # if item_type_id > 30:
    #     break

tensor([1, 2])


100%|██████████| 2/2 [00:00<00:00, 27.97it/s]


In [6]:
item_type_res_dict['item_type_id'] = list(dataset.id2token('item_type_recommend', item_type_res_dict['item_type_id']))
item_type_res = pd.DataFrame(item_type_res_dict)
item_type_res.head()

,item_type_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10
0,POI_WAIMAI,26013,23133,1327,1553,0.2183,0.1255,0.1473,0.2183,0.0218
1,POI_XIUYU,1314,1018,119,177,0.1638,0.1406,0.1462,0.1638,0.0164


In [8]:
item_type_res['total_inter_num'].sum()

343468

In [9]:
item_type_res['train_inter_num'].sum() + item_type_res['valid_inter_num'].sum() + item_type_res['test_inter_num'].sum()

343468

In [10]:
item_type_res.sort_values(by='recall@10', ascending=False).head(50)

,item_type_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10
0,POI_WAIMAI,328981,298111,14290,16580,0.2000,0.1247,0.1424,0.2000,0.0200
1,POI_XIUYU,14487,11859,1088,1540,0.1916,0.1503,0.1604,0.1916,0.0192


In [32]:
item_type_df = pd.read_csv('~/item_type.txt', sep='\t')
print(len(item_type_df))
item_type_df = item_type_df[['mt_item_type_id', 'mt_item_type_name']]
item_type_df.columns = ['item_type_id', 'item_type_name']
item_type_df.head()

1226


,city_id,city_name
0,936,西华
1,842,盘州市
2,287,潮州
3,1195,福安
4,1203,东明县


In [33]:
print(len(item_type_res))

1178


In [34]:
new_item_type_res = pd.merge(item_type_res, item_type_df, how='left', on='item_type_id')
print(len(new_item_type_res))
new_item_type_res.head()

1178


,city_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10,city_name
0,482,3133,2929,99,105,0.1333,0.0554,0.0738,0.1333,0.0133,腾冲
1,324,15195,14355,411,429,0.0769,0.0232,0.0356,0.0769,0.0077,安顺
2,30,627762,593706,17075,16981,0.0775,0.0353,0.0451,0.0775,0.0077,深圳
3,73,248466,233518,7483,7465,0.0579,0.0283,0.0352,0.0579,0.0058,郑州
4,99,195176,185833,4672,4671,0.0612,0.0282,0.0359,0.0612,0.0061,南宁


In [37]:
new_item_type_res.sort_values(by='item_type_id', ignore_index=True, inplace=True)

In [38]:
new_item_type_res.head()

,city_id,total_inter_num,train_inter_num,valid_inter_num,test_inter_num,recall@10,mrr@10,ndcg@10,hit@10,precision@10,city_name
0,1,595922,555699,20158,20065,0.0778,0.0378,0.0471,0.0778,0.0078,北京
1,10,506161,476555,14815,14791,0.0721,0.0336,0.0425,0.0721,0.0072,上海
2,20,608105,574994,16567,16544,0.0800,0.0377,0.0476,0.0800,0.0080,广州
3,30,627762,593706,17075,16981,0.0775,0.0353,0.0451,0.0775,0.0077,深圳
4,40,204110,189901,7097,7112,0.0697,0.0338,0.0422,0.0697,0.0070,天津


In [39]:
new_item_type_res.to_csv('~/item_type_result.csv', sep='\t', index=False)